Função que Grava o Dataframe no Data Lake criando uma Delta Table 

In [ ]:


def storeInBlobDatabricks(df,path,schema,tabela,format_write,modo):
    try:
        df = df.withColumn("dt_folder", current_date().cast("date"))
        (df.write.format(format_write)
        .mode(modo)
        .partitionBy('dt_folder')
        .option("mergeSchema", "true")
        .option("path", path)
        .saveAsTable(schema+tabela))
        return True
    except Exception as ex:
        return ex

Função que remove os espaços de todos os campos tipo string no Dataframe

In [ ]:

def RemovSpaces(df):
    df_tr = (
        df.select(
            *(trim(col(colu)).alias(colu) for colu, ty in df.dtypes if ty.split('(')[0] in ('string')),
            *(col(lun).alias(lun)for lun, tpy in df.dtypes if tpy.split('(')[0] != 'string')
        )
    )
    
    return df_tr

Função que Grava o dataframe no Synapse por PolyBase, utilizando como método de autenticação Managed Service Identity seguindo as práticas de segurança - KeyVault


In [ ]:
def grava_no_dw(df, tabela, dw_conexao, temp):
    spark.conf.set("fs.azure.account.auth.type.meudatalake.dfs.core.windows.net", "OAuth")
    spark.conf.set("fs.azure.account.oauth.provider.type.meudatalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
    spark.conf.set("fs.azure.account.oauth2.client.id.meudatalake.dfs.core.windows.net", dbutils.secrets.get(scope = "key-vault", key = "clientid"))
    spark.conf.set("fs.azure.account.oauth2.client.secret.meudatalake.dfs.core.windows.net", dbutils.secrets.get(scope = "key-vault", key = "secret"))
    spark.conf.set("fs.azure.account.oauth2.client.endpoint.meudatalake.dfs.core.windows.net", "https://login.microsoftonline.com/"+dbutils.secrets.get(scope = "key-vault", key = "DiretorioId")+"/oauth2/token")
    
    df.write.format("com.databricks.spark.sqldw")
    .option("url", dw_conexao)
    .option("useAzureMSI", "true")
    .option("dbTable", tabela)
    .mode("append")
    .option("tempDir", temp)
    .save()